_Alex Malz (NYU)_

# Quantifying support between COSMOS and zCOSMOS

The data: [zCOSMOS](http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J%2FApJS%2F184%2F218) and [COSMOS](http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=II%2F284).

In [ ]:
import astropy as ap
from astropy.io import fits
from astropy.table import Table
import csv

import numpy as np
import pandas as pd
import scipy.stats as sps

import timeit

import causalinference

In [ ]:
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['savefig.dpi'] = 250
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['savefig.bbox'] = 'tight'

import matplotlib.pyplot as plt
%matplotlib inline
import corner

## Exploratory data analysis

Read in the data produced by [CDS XMatch](cdsxmatch.u-strasbg.fr/xmatch) set to 5".

In [ ]:
with open('1511124733287A.csv', 'rb') as csvfile:
    xmatchreader = csv.reader(csvfile)
    tuples = (line for line in xmatchreader)
    col_names = tuples.next()
    print(col_names)
    n_cols = len(col_names)
    col_inds = range(n_cols)
    xmatch = [[pair[k] for k in col_inds] for pair in tuples]
test_data = pd.DataFrame(xmatch, columns=col_names)

How many pairs are found?

In [ ]:
original_xmatch = test_data[['zCOSMOS', 'COSMOS']].astype(int)
print(original_xmatch.shape)

Remove duplicate matches -- I should replace this with dropping the duplicate with larger `'angDist'` instead of dropping both.  How many were duplicates?

In [ ]:
unique_xmatch = original_xmatch.drop_duplicates(subset=['zCOSMOS'], keep=False)
xmatch = unique_xmatch.drop_duplicates(subset=['COSMOS'], keep=False)
print(xmatch.shape)

FITS file of both spectroscopic and photometric data -- I have no memory of how I got both of these in one file.
Warning: this step is slow because this is the whole dataset!

In [ ]:
with fits.open('asu.fit') as hdulist:
    print(hdulist.info())
    spec_header = hdulist[1].header
    phot_header = hdulist[2].header
    spec_data = pd.DataFrame(hdulist[1].data)
    phot_data = pd.DataFrame(hdulist[2].data)

What do we know about zCOSMOS galaxies?

In [ ]:
print(spec_header)
print(spec_data.columns)

What do we know about COSMOS galaxies?

In [ ]:
print(phot_header)
print(phot_data.columns)

Let's merge zCOSMOS columns to zCOSMOS galaxies that appear in the cross-match and COSMOS columns to COSMOS galaxies that appear in the cross-match.

In [ ]:
partial = xmatch.merge(spec_data, on='zCOSMOS', how="outer")
print(partial.columns)
combo = partial.merge(phot_data, on='COSMOS', how='outer')
print(combo.columns)

Now let's construct separate dataframes for the COSMOS and zCOSMOS data for all zCOSMOS galaxies and the COSMOS data for COSMOS galaxies that aren't also zCOSMOS galaxies.

In [ ]:
spec_samp = xmatch.merge(spec_data, on='zCOSMOS', how="left").merge(phot_data, on='COSMOS', how='left')
phot_samp = phot_data[(~phot_data['COSMOS'].isin(xmatch['COSMOS']))]

In [ ]:
print(len(spec_data), len(phot_data))
print(len(spec_samp), len(phot_samp))

In [ ]:
plt.scatter(spec_samp['z'], spec_samp['zphot'], color='k', marker='.', s=5, alpha=0.25)
plt.xlabel('z')
plt.ylabel('zphot')
plt.savefig('scatter.png')

Those `'z'=0.0` galaxies are definitely not physical.  Let's check what their quality flag was.

In [ ]:
spec_samp['CClass'][spec_samp['z'] == 0.].value_counts().plot(kind='bar')

Oh yeah, they were overwhelmingly flagged as being bad observations, so let's get rid of them.  Interestingly, the galaxies assigned `'zphot'=0.0` weren't all bad.

In [ ]:
spec_samp['CClass'][spec_samp['zphot'] == 0.].value_counts().plot(kind='bar')

Let's drop all the galaxies flagged as being bad observations to see if it fixes the problem

In [ ]:
spec_samp_cleaner_z = spec_samp[spec_samp['CClass'] != 0.]
phot_samp_cleaner_z = phot_samp

In [ ]:
print(spec_samp_cleaner_z.shape)
print(phot_samp_cleaner_z.shape)

Okay, let's see if cutting out the bad observations removed all those definitely wrong zeros.

In [ ]:
plt.scatter(spec_samp_cleaner_z['z'], spec_samp_cleaner_z['zphot'], marker='.', s=5, alpha=0.25, color='k')
plt.xlabel('z')
plt.ylabel('zphot')
plt.savefig('scatter_cleaner.png')

Nope, they're still there!  Let's just cut them out and hope for the best.

In [ ]:
spec_samp_clean_z = spec_samp_cleaner_z[spec_samp_cleaner_z['z'] != 0.]#spec_samp[spec_samp['z'] != 0.][spec_samp['zphot'] != 0.]
phot_samp_clean_z = phot_samp_cleaner_z[phot_samp_cleaner_z['zphot'] != 0.]#phot_samp[phot_samp['zphot'] != 0.]

In [ ]:
plt.scatter(spec_samp_clean_z['z'], spec_samp_clean_z['zphot'], marker='.', s=5, alpha=0.25, color='k')
plt.xlabel('z')
plt.ylabel('zphot')
plt.savefig('scatter_cleanest.png')

Now we can check the distributions of the zCOSMOS+COSMOS `'z'` and `'zphot'` as well as the COSMOS-only `'zphot'`.  Do they have common support?

In [ ]:
z_range = np.linspace(0., 3.5, 50)
plt.hist(spec_samp_clean_z['z'], bins=z_range, normed=True, color='k', alpha=0.25, label='z (spec sample)', histtype='step', linewidth=2)
plt.hist(spec_samp_clean_z['zphot'], bins=z_range, normed=True, color='b', alpha=0.25, label='zphot (spec sample)', histtype='step', linewidth=2)
plt.hist(phot_samp_clean_z['zphot'], bins=z_range, normed=True, color='r', alpha=0.25, label='zphot (phot sample)', histtype='step', linewidth=2)
plt.legend(loc='upper right')
plt.ylabel('normalized frequency')
plt.xlabel('z')
plt.savefig('hist.png')

We also know that the magnitudes are set to 0. when the observation is bad, so let's filter out all of those, too.

In [ ]:
# this is not an efficient way to do this!
spec_samp_clean_zmags = spec_samp_clean_z[spec_samp_clean_z['Bmag'] != 0.][spec_samp_clean_z['Vmag'] != 0.][spec_samp_clean_z['gmag'] != 0.][spec_samp_clean_z['rmag'] != 0.][spec_samp_clean_z['imag'] != 0.][spec_samp_clean_z['zmag'] != 0.]
phot_samp_clean_zmags = phot_samp_clean_z[phot_samp_clean_z['Bmag'] != 0.][phot_samp_clean_z['Vmag'] != 0.][phot_samp_clean_z['gmag'] != 0.][phot_samp_clean_z['rmag'] != 0.][phot_samp_clean_z['imag'] != 0.][phot_samp_clean_z['zmag'] != 0.]

print(len(spec_samp_clean_zmags))
print(len(phot_samp_clean_zmags))

I want to make a really sick plot showing the distributions of data for the zCOSMOS and COSMOS datasets.

In [ ]:
def density_estimation(m1, m2):
    X, Y = np.mgrid[min(m1):max(m1):100j, min(m2):max(m2):100j]                                                     
    positions = np.vstack([X.ravel(), Y.ravel()])                                                       
    values = np.vstack([m1, m2])                                                                        
    kernel = sps.gaussian_kde(values)                                                             
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z

def mycorner(data, keys, colors, maps, lims=None, pre_densities=None, filename='plot.pdf'):
    ncol = len(keys)
    fig = plt.figure(figsize=(ncol*5, ncol*5))
    ax = [[fig.add_subplot(ncol, ncol, ncol * i + j + 1) for j in range(i+1)] for i in range(ncol)]
    for k in range(len(data)):
        datum = data[k]
        npoints = len(datum)
        for i in range(ncol):
            for j in range(i+1):
                if i == j:
                    ax[i][j].hist(datum[keys[i]], bins=50, histtype='step', linewidth=2, normed=True, alpha=0.75, color=colors[k])
                    ax[i][j].set_xlabel(keys[i])
                else:
                    if npoints > 1e4:
                        ax[i][j].hist2d(datum[keys[i]], datum[keys[j]], bins=(100, 100), normed=True, cmap=maps[k], alpha=0.5)
                    else:
                        if pre_densities is None:
                            x, y, z = density_estimation(datum[keys[i]], datum[keys[j]])
                        else:
                            (x, y, z) = pre_densities[i][j]
                        ax[i][j].contour(x, y, z, cmap=plt.get_cmap(maps[k]) , alpha=0.5)
                    ax[i][j].set_xlabel(keys[i])
                    ax[i][j].set_ylabel(keys[j])
                    if lims is not None:
                        ax[i][j].set_xlim(lims)
                        ax[i][j].set_ylim(lims)
    fig.savefig(filename, dpi=100)
    return
# replace with 2d histogram for speed

Here it is, with blue for zCOSMOS galaxies and red for COSMOS-only galaxies.

In [ ]:
post_densities = mycorner([spec_samp_clean_zmags, phot_samp_clean_zmags], ['Bmag', 'Vmag', 'gmag', 'rmag','imag', 'zmag', 'zphot'], ['b', 'r'], ['Blues', 'Reds'], filename='big_corner_coarse.png')

That was using the magnitudes, which are log-fluxes.  We know that colors, the differences between adjacent magnitudes, are more informative because they're resistant to the dimming due to distance alone, i.e. they have more information about the galaxy's redshift if it were nearer to us.  This controls for some galaxies just being brighter than others, so we compare those that would otherwise have a similar appearance aside from brightness.

In [ ]:
covars = ['B-V', 'V-g', 'g-r', 'r-i', 'i-z']

def make_colors(data):
    data['B-V'] = data['Bmag'] - data['Vmag']
    data['V-g'] = data['Vmag'] - data['gmag']
    data['g-r'] = data['gmag'] - data['rmag']
    data['r-i'] = data['rmag'] - data['imag']
    data['i-z'] = data['imag'] - data['zmag']
    return(data)

In [ ]:
make_colors(spec_samp_clean_zmags)
make_colors(phot_samp_clean_zmags)

Let's make that plot again!

In [ ]:
post_densities = mycorner([spec_samp_clean_zmags, phot_samp_clean_zmags], ['B-V', 'V-g', 'g-r', 'r-i', 'i-z', 'zphot'], ['b', 'r'], ['Blues', 'Reds'], filename='color_corner.png')

Sweet, this looks good, so we can save the cleaned data and use that from now on.

In [ ]:
spec_samp_clean_zmags.to_pickle('spec_samp.pkl')
phot_samp_clean_zmags.to_pickle('phot_samp.pkl')

# Ignore after this point!

The colors are better predictors of redshift than the magnitudes.

In [ ]:
# covars = ['B-V', 'V-g', 'g-r', 'r-i', 'i-z']

# print(spec_samp_clean_zmags[covars])

In [ ]:
# print(len(spec_samp_clean_zmags))
# corner.corner(spec_samp_clean_zmags[['umag', 'gmag','rmag','imag', 'zmag']])

In [ ]:
# print(len(phot_samp_clean_zmags))
# corner.corner(phot_samp_clean_zmags[['umag', 'gmag','rmag','imag', 'zmag']])

In [ ]:
# print(spec_data.columns, phot_data.columns)